In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from pandas.plotting import scatter_matrix
from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as lm
%matplotlib inline


In [2]:
df = pd.read_csv('train.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [6]:
df.corr()['Survived'].sort_values(ascending = False)

Survived       1.000000
Fare           0.257307
Parch          0.081629
PassengerId   -0.005007
SibSp         -0.035322
Age           -0.077221
Pclass        -0.338481
Name: Survived, dtype: float64

In [7]:
df.Survived.value_counts() ##Base Model is 549/(549 + 342) = .616

0    549
1    342
Name: Survived, dtype: int64

In [8]:
df.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [43]:
df.Ticket.value_counts()

347082              7
1601                7
CA. 2343            7
347088              6
CA 2144             6
3101295             6
S.O.C. 14879        5
382652              5
113781              4
LINE                4
4133                4
W./C. 6608          4
113760              4
PC 17757            4
17421               4
349909              4
347077              4
2666                4
19950               4
363291              3
PC 17582            3
29106               3
PC 17755            3
248727              3
110413              3
230080              3
13502               3
PC 17760            3
347742              3
PC 17572            3
                   ..
349205              1
349240              1
STON/O2. 3101283    1
693                 1
PC 17482            1
113051              1
113796              1
323592              1
2669                1
F.C.C. 13528        1
350029              1
112059              1
A./5. 3235          1
19988               1
345770    

In [12]:
pd.get_dummies(df[['Embarked', 'Sex']])

,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,0,0,1,0,1
1,1,0,0,1,0
2,0,0,1,1,0
3,0,0,1,1,0
4,0,0,1,0,1
5,0,1,0,0,1
6,0,0,1,0,1
7,0,0,1,0,1
8,0,0,1,1,0
9,1,0,0,1,0


In [15]:
df['Name'][0].split()

['Braund,', 'Mr.', 'Owen', 'Harris']

In [20]:
titles = []
for i in range(len(df['Name'])):
    nl = df['Name'][i].split()
    if 'Mr.' in nl:
        titles.append('Mr.')
    elif 'Miss.' in nl:
        titles.append('Miss.')
    else:
        titles.append('Master.')

In [22]:
df['title'] = titles

In [24]:
dummies = pd.get_dummies(df[['Embarked', 'Sex', 'title']]) ##all categorical variables that could matter in some capacity

In [68]:
full = pd.concat([df, dummies], axis = 1)

In [69]:
full.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Embarked,title,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,title_Master.,title_Miss.,title_Mr.
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,S,Mr.,0,0,1,0,1,0,0,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,C,Master.,1,0,0,1,0,1,0,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,S,Miss.,0,0,1,1,0,0,1,0
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,S,Master.,0,0,1,1,0,1,0,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,S,Mr.,0,0,1,0,1,0,0,1


In [70]:
full.corr()['Survived'].sort_values(ascending = False)

Survived         1.000000
Sex_female       0.543351
title_Master.    0.338476
title_Miss.      0.327093
Fare             0.257307
Embarked_C       0.168240
Parch            0.081629
Embarked_Q       0.003650
PassengerId     -0.005007
SibSp           -0.035322
Age             -0.077221
Embarked_S      -0.155660
Pclass          -0.338481
Sex_male        -0.543351
title_Mr.       -0.549199
Name: Survived, dtype: float64

In [71]:
full.info() #looks like age is missing values
age_median = full.Age.median()
full['Age'] = full.Age.fillna(age_median)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 21 columns):
PassengerId      891 non-null int64
Survived         891 non-null int64
Pclass           891 non-null int64
Name             891 non-null object
Sex              891 non-null object
Age              714 non-null float64
SibSp            891 non-null int64
Parch            891 non-null int64
Ticket           891 non-null object
Fare             891 non-null float64
Cabin            204 non-null object
Embarked         889 non-null object
title            891 non-null object
Embarked_C       891 non-null uint8
Embarked_Q       891 non-null uint8
Embarked_S       891 non-null uint8
Sex_female       891 non-null uint8
Sex_male         891 non-null uint8
title_Master.    891 non-null uint8
title_Miss.      891 non-null uint8
title_Mr.        891 non-null uint8
dtypes: float64(2), int64(5), object(6), uint8(8)
memory usage: 97.5+ KB


In [102]:
X = full.drop(columns = ['Survived','Sex_male', 'PassengerId', 'title_Mr.', 'Embarked_C', 'Sex', 'title', 'Cabin', 'Name', 'Embarked', 'Ticket'])
#Remove response variable, the string categorical variables, and one of the dummy categories classified from the initial string
X

,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_female,title_Master.,title_Miss.
0,3,22.0,1,0,7.2500,0,1,0,0,0
1,1,38.0,1,0,71.2833,0,0,1,1,0
2,3,26.0,0,0,7.9250,0,1,1,0,1
3,1,35.0,1,0,53.1000,0,1,1,1,0
4,3,35.0,0,0,8.0500,0,1,0,0,0
5,3,28.0,0,0,8.4583,1,0,0,0,0
6,1,54.0,0,0,51.8625,0,1,0,0,0
7,3,2.0,3,1,21.0750,0,1,0,1,0
8,3,27.0,0,2,11.1333,0,1,1,1,0
9,2,14.0,1,0,30.0708,0,0,1,1,0


In [103]:
y = full.Survived
type(y)

pandas.core.series.Series

In [104]:
X.head()
X.shape

(891, 10)

In [105]:
X.info() #looks like some age are missing

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Pclass           891 non-null int64
Age              891 non-null float64
SibSp            891 non-null int64
Parch            891 non-null int64
Fare             891 non-null float64
Embarked_Q       891 non-null uint8
Embarked_S       891 non-null uint8
Sex_female       891 non-null uint8
title_Master.    891 non-null uint8
title_Miss.      891 non-null uint8
dtypes: float64(2), int64(3), uint8(5)
memory usage: 39.2 KB


In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)

In [107]:
full.corr()['Survived']

PassengerId     -0.005007
Survived         1.000000
Pclass          -0.338481
Age             -0.064910
SibSp           -0.035322
Parch            0.081629
Fare             0.257307
Embarked_C       0.168240
Embarked_Q       0.003650
Embarked_S      -0.155660
Sex_female       0.543351
Sex_male        -0.543351
title_Master.    0.338476
title_Miss.      0.327093
title_Mr.       -0.549199
Name: Survived, dtype: float64

In [134]:
model = lm.LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [140]:
X_test

,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_female,title_Master.,title_Miss.
709,3,28.0,1,1,15.2458,0,0,0,1,0
439,2,31.0,0,0,10.5000,0,1,0,0,0
840,3,20.0,0,0,7.9250,0,1,0,0,0
720,2,6.0,0,1,33.0000,0,1,1,0,1
39,3,14.0,1,0,11.2417,0,0,1,0,1
290,1,26.0,0,0,78.8500,0,1,1,0,1
300,3,28.0,0,0,7.7500,1,0,1,0,1
333,3,16.0,2,0,18.0000,0,1,0,0,0
208,3,16.0,0,0,7.7500,1,0,1,0,1
136,1,19.0,0,2,26.2833,0,1,1,0,1


In [135]:
model.predict(X_test)

array([0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 1, 0], dtype=int64)

In [136]:
model.score(X_test, y_test) ## does about 20% better than base model performance of 61.6%

0.8101694915254237

In [137]:
model.coef_

array([[-0.68818243, -0.02576249, -0.36500498, -0.27925771,  0.00578152,
        -0.144344  , -0.54206936,  1.77298728,  1.71801348,  0.60544203]])

In [138]:
model.intercept_

array([1.11797972])

AttributeError: 'LogisticRegression' object has no attribute 'summary'